# Topic modeling

In [1]:
# !pip install -qU pip wheel
# !pip install -qU gensim tqdm pandas nltk numpy
# import nltk
# nltk.download('stopwords')

In [2]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import gensim
import random
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
TOTAL_TOPICS = 25

In [4]:
# corpus
path_corpus=os.path.expanduser('~/ppa_data/solrcorpus')
path_metadata = os.path.join(path_corpus, 'metadata.csv')
path_minimal = os.path.join(path_corpus, 'minimal.jsonl')
path_texts = os.path.join(path_corpus, 'texts')
path_minimal_numlines=None

In [5]:
# Read metadata
df_metadata = pd.read_csv(path_metadata).fillna('').set_index('work_id')
df_metadata

,source_id,group_id_s,source_t,source_url,title,sort_title,pub_date,pub_place,publisher,enumcron,...,cluster_id_s,item_type,order,work_type_s,last_modified,_version_,first_page_i,book_journal_s,subtitle,notes
work_id,,,,,,,,,,,,,,,,,,,,,
dul1.ark:|13960|t6d23816n,dul1.ark:/13960/t6d23816n,dul1.ark:/13960/t6d23816n,['HathiTrust'],https://hdl.handle.net/2027/dul1.ark:/13960/t6...,"Lyrical ballads, with other poems: in two volu...","Lyrical ballads, with other poems: in two volu...",1802.0,Philadelphia,James Humphreys,v.1-2,...,dul1.ark:/13960/t6d23816n,work,0,full-work,2023-02-02T15:00:45.957Z,1756731849797795847,,,,
dul1.ark:|13960|t6d23816n-p11,dul1.ark:/13960/t6d23816n,dul1.ark:/13960/t6d23816n-p11,['HathiTrust'],https://hdl.handle.net/2027/dul1.ark:/13960/t6...,Preface [to Lyrical Ballads],Preface [to Lyrical Ballads],1802.0,Philadelphia,James Humphreys,v.1,...,dul1.ark:/13960/t6d23816n-p11,work,0,excerpt,2023-02-14T19:59:25.780Z,1757837803736006658,11.0,"Lyrical ballads, with other poems: in two volu...",,
inu.32000006890968,inu.32000006890968,inu.32000006890968,['HathiTrust'],https://hdl.handle.net/2027/inu.32000006890968,"The mystical poets of the English church,","mystical poets of the English church,",1919.0,London,Society for Promoting Christian Knowledge,,...,osmondmystical,work,0,full-work,2023-02-14T19:34:42.875Z,1757836248763858945,,,,
hvd.hn5bic,hvd.hn5bic,hvd.hn5bic,['HathiTrust'],https://hdl.handle.net/2027/hvd.hn5bic,An analysis of the derivative words in the Eng...,analysis of the derivative words in the Englis...,1841.0,New York,Clement & Packard,,...,townanalysis,work,0,full-work,2023-02-14T19:59:31.530Z,1757837809744347137,,,"or, A key to their precise analytic definition...",
CW0100721039,CW0100721039,CW0100721039,['Gale'],https://link.gale.com/apps/doc/CW0100721039/EC...,A new and general biographical dictionary,new and general biographical dictionary contai...,1798.0,London,"G.G. and J. Robinson, J. Johnson, J. Nichols, ...",14,...,CW0100721039,work,0,full-work,2023-02-14T19:34:42.875Z,1757836248776441856,,,containing an historical and critical account ...,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
njp.32101076530979,njp.32101076530979,njp.32101076530979,['HathiTrust'],https://hdl.handle.net/2027/njp.32101076530979,"Essays philosophical and moral, historical and...","Essays philosophical and moral, historical and...",1799.0,London,G.G. and J. Robinson,v.2,...,njp.32101076530979,work,0,full-work,2023-02-02T15:00:44.887Z,1756731848685256707,,,,
njp.32101076530979-p464,njp.32101076530979,njp.32101076530979-p476,['HathiTrust'],https://hdl.handle.net/2027/njp.32101076530979,On Shakespeare,On Shakespeare,1799.0,London,G.G. and J. Robinson,v.2,...,njp.32101076530979-p476,work,0,excerpt,2023-02-14T19:59:23.288Z,1757837801082060801,476.0,"Essays philosophical and moral, historical and...",,
njp.32101076530979-p482,njp.32101076530979,njp.32101076530979-p494,['HathiTrust'],https://hdl.handle.net/2027/njp.32101076530979,On Stile and Versification,On Stile and Versification,1799.0,London,G.G. and J. Robinson,v.2,...,njp.32101076530979-p494,work,0,excerpt,2023-02-14T19:59:23.288Z,1757837801083109376,494.0,"Essays philosophical and moral, historical and...",,


In [6]:
import nltk
from nltk.corpus import stopwords as stops
stopwords = set(stops.words('english')) | {'one','may'}
# stopwords

In [7]:
import orjson

In [8]:
import jsonlines,orjson

def iter_id_tokens(min_toks=50):
    global path_minimal_numlines
    if path_minimal_numlines == None:
        with open(path_minimal) as f:
            path_minimal_numlines = sum(1 for line in tqdm(f,desc='Getting number of lines',position=0))

    with open(path_minimal) as f:
        for ln in tqdm(f,total=path_minimal_numlines,desc='Iterating over jsonl',position=0):
            try:
                d=orjson.loads(ln)
                yield (d['id'], d['toks'])
            except Exception:
                pass
    
def iter_tokens():
    for id,toks in iter_id_tokens():
        yield toks


next(iter_id_tokens())

Getting number of lines: 2097745it [00:07, 268364.55it/s]
Iterating over jsonl:   0%|          | 0/2097745 [00:00<?, ?it/s]


('dul1.ark:|13960|t6d23816n_2',
 ['frances',
  'e',
  'bennett',
  'i',
  'm',
  'n',
  'womn',
  "'r",
  'j',
  'v',
  'u',
  'duke',
  'university',
  'library',
  'treasure',
  'oom',
  'the',
  'coleridge',
  'collection'])

# Transforming corpus into bag of words vectors

We can now perform feature engineering by leveraging a simple Bag of Words
model.

In [9]:
fn=os.path.join(path_corpus,'data.gensim.dictionary.pkl')
if not os.path.exists(fn):
    dictionary = gensim.corpora.Dictionary(iter_tokens())
    dictionary.save(fn)
else:
    dictionary = gensim.corpora.Dictionary.load(fn)
len(dictionary)

2023-11-29 12:51:29,900 : INFO : loading Dictionary object from /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.dictionary.pkl
2023-11-29 12:51:31,244 : INFO : Dictionary lifecycle event {'fname': '/Users/ryanheuser/ppa_data/solrcorpus/data.gensim.dictionary.pkl', 'datetime': '2023-11-29T12:51:31.244873', 'gensim': '4.3.2', 'python': '3.12.0 (main, Nov  7 2023, 18:55:06) [Clang 15.0.0 (clang-1500.0.40.1)]', 'platform': 'macOS-14.0-x86_64-i386-64bit', 'event': 'loaded'}


2027040

In [10]:
dictionary.filter_extremes(no_above=.9, no_below=25)
# dictionary.filter_extremes(keep_n=50000)
len(dictionary)

2023-11-29 12:51:32,881 : INFO : discarding 1927040 tokens: [('the', 1992801), ('womn', 89), ('and', 1940820), ('atreet', 24), ('elpjjia', 1), ('genium', 130), ('in', 1951311), ('of', 1978646), ('papiniane', 3), ('14j', 41)]...
2023-11-29 12:51:32,881 : INFO : keeping 100000 tokens which were in no less than 25 and no more than 1887970 (=90.0%) documents
2023-11-29 12:51:33,371 : INFO : resulting dictionary: Dictionary<100000 unique tokens: ["'r", 'bennett', 'coleridge', 'collection', 'duke']...>


100000

In [106]:
# Transforming corpus into bag of words vectors
from collections import Counter

def iter_corpus(max_pages_per_work=25,min_words_per_page=25):
    idnumpages=Counter()
    for i,(page_id,page_toks) in enumerate(iter_id_tokens()):
        page_toks = [tok for tok in page_toks if tok not in stopwords and len(tok)>3]
        docid=page_id.split('_')[0]
        id=df_metadata.loc[docid].cluster_id_s
        if not min_words_per_page or len(page_toks) >= min_words_per_page:
            idnumpages[id]+=1
            if not max_pages_per_work or idnumpages[id]<=max_pages_per_work:
                yield {'work_id':docid,'work_cluster':id,'page_id':page_id,'page_toks':page_toks}
        

def iter_corpus_tokens():
    for d in iter_corpus():
        yield dictionary.doc2bow(d['page_toks'])

next(iter_corpus())
# next(iter_corpus_tokens())

Iterating over jsonl:   0%|          | 4/2097745 [00:00<25:41, 1361.01it/s]


{'work_id': 'dul1.ark:|13960|t6d23816n',
 'work_cluster': 'dul1.ark:/13960/t6d23816n',
 'page_id': 'dul1.ark:|13960|t6d23816n_9',
 'page_toks': ['contents',
  'expostulation',
  'reply',
  'page',
  'tables',
  'turned',
  'evening',
  'scene',
  'subject',
  'animal',
  'tranquillity',
  'decay',
  'sketch',
  'complaint',
  'forsaken',
  'indian',
  'woman',
  'last',
  'flock',
  'lines',
  'left',
  'upon',
  'seat',
  'tree',
  'stands',
  'near',
  'lake',
  'esthwaite',
  'foster',
  "mother's",
  'tale',
  'goody',
  'blake',
  'harry',
  'gill',
  'thorn',
  'seven',
  'anecdote',
  'fathers',
  'lines',
  'written',
  'small',
  'distance',
  'house',
  'sent',
  'little',
  'person',
  'addressed',
  'female',
  'vagrant',
  'dungeon',
  'simon',
  'huntsman',
  'lines',
  'written',
  'early',
  'spring',
  'nightingale',
  'written',
  'april',
  'lines',
  'written',
  'near',
  'richmond',
  'upon',
  'thames',
  'idiot',
  'love',
  'mother',
  'ancient',
  'manner',
  

In [107]:
# Transforming corpus into bag of words vectors
def get_gensim_corpus(force=False):
    fn=os.path.join(path_corpus,'data.gensim.corpus.pkl')
    if force or not os.path.exists(fn):
        gensim.corpora.MmCorpus.serialize(
            fname=fn,
            corpus=iter_corpus_tokens(),
            id2word=dictionary,
        )

    corpus = gensim.corpora.MmCorpus(fn)
    return corpus

In [108]:
corpus = get_gensim_corpus(force=True)

2023-11-29 13:57:28,636 : INFO : storing corpus in Matrix Market format to /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl
2023-11-29 13:57:28,646 : INFO : saving sparse matrix to /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl
Iterating over jsonl: 100%|██████████| 2097745/2097745 [08:31<00:00, 4104.93it/s]
2023-11-29 14:05:59,681 : INFO : saved 112189x100000 matrix, density=0.103% (11524551/11218900000)
2023-11-29 14:05:59,683 : INFO : saving MmCorpus index to /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl.index
2023-11-29 14:05:59,712 : INFO : loaded corpus index from /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl.index
2023-11-29 14:05:59,713 : INFO : initializing cython corpus reader from /Users/ryanheuser/ppa_data/solrcorpus/data.gensim.corpus.pkl
2023-11-29 14:05:59,715 : INFO : accepted corpus with 112189 documents, 100000 features, 11524551 non-zero entries


In [109]:
def topic_model(num_topics=TOTAL_TOPICS, force=False):
    fn=os.path.join(path_corpus,f'data.gensim.lda_model.ntopic={num_topics}.pkl')
    corpus_sample = corpus

    if force or not os.path.exists(fn):
        lda_model = gensim.models.ldamulticore.LdaMulticore(
            corpus=corpus_sample, 
            id2word=dictionary,
            num_topics=num_topics,
            workers=7,
            # chunksize=100,
            # alpha='auto', 
            # eta='auto', 
            random_state=42,
            iterations=50, 
            passes=20, 
            # eval_every=None
        )
        lda_model.save(fn)
    else:
        lda_model = gensim.models.LdaModel.load(fn)
    return lda_model

In [110]:
lda_model = topic_model(50, force=True)

2023-11-29 14:06:26,956 : INFO : using symmetric alpha at 0.02
2023-11-29 14:06:26,958 : INFO : using symmetric eta at 0.02
2023-11-29 14:06:26,979 : INFO : using serial LDA version on this node
2023-11-29 14:06:27,600 : INFO : running online LDA training, 50 topics, 20 passes over the supplied corpus of 112189 documents, updating every 14000 documents, evaluating every ~112189 documents, iterating 50x with a convergence threshold of 0.001000
2023-11-29 14:06:27,602 : INFO : training LDA model using 7 processes
2023-11-29 14:06:38,193 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #2000/112189, outstanding queue size 1
2023-11-29 14:06:38,490 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #4000/112189, outstanding queue size 2
2023-11-29 14:06:38,709 : INFO : PROGRESS: pass 0, dispatched chunk #2 = documents up to #6000/112189, outstanding queue size 3
2023-11-29 14:06:38,975 : INFO : PROGRESS: pass 0, dispatched chunk #3 = documents up to #8000/1121

In [111]:
topics_coherences = lda_model.top_topics(corpus, topn=20)
avg_coherence_score = np.mean([item[1] for item in topics_coherences])
print('Avg. Coherence Score:', avg_coherence_score)

2023-11-29 14:22:52,755 : INFO : CorpusAccumulator accumulated stats from 1000 documents
2023-11-29 14:22:52,850 : INFO : CorpusAccumulator accumulated stats from 2000 documents
2023-11-29 14:22:52,945 : INFO : CorpusAccumulator accumulated stats from 3000 documents
2023-11-29 14:22:53,034 : INFO : CorpusAccumulator accumulated stats from 4000 documents
2023-11-29 14:22:53,140 : INFO : CorpusAccumulator accumulated stats from 5000 documents
2023-11-29 14:22:53,233 : INFO : CorpusAccumulator accumulated stats from 6000 documents
2023-11-29 14:22:53,328 : INFO : CorpusAccumulator accumulated stats from 7000 documents
2023-11-29 14:22:53,434 : INFO : CorpusAccumulator accumulated stats from 8000 documents
2023-11-29 14:22:53,530 : INFO : CorpusAccumulator accumulated stats from 9000 documents
2023-11-29 14:22:53,626 : INFO : CorpusAccumulator accumulated stats from 10000 documents
2023-11-29 14:22:53,723 : INFO : CorpusAccumulator accumulated stats from 11000 documents
2023-11-29 14:22:53

Avg. Coherence Score: -2.8713920960844086


In [112]:
def get_topic_terms(lda_model, topn=25):
    return {
        topicid:', '.join([dictionary[i] for i,j in lda_model.get_topic_terms(topicid,topn=topn)])
        for topicid in range(lda_model.num_topics)
    }

In [113]:
topicwords = get_topic_terms(lda_model)
topicwords

{0: 'words, word, meaning, spelling, would, used, read, pronunciation, language, thus, sense, make, every, reading, must, many, different, without, written, dictionary, another, often, speak, first, correct',
 1: 'literature, century, history, time, english, great, work, many, literary, years, much, first, period, writers, works, though, even, modern, early, among, latin, still, written, could, greek',
 2: 'university, library, california, cornell, berkeley, univ, accessory, vers, sity, polo, adnominal, expletive, έστι, grocer, univers, angelus, pickwick, sain, hodge, forde, city, gift, ordo, chevy, doublet',
 3: "state, pref, tion, ness, quality, ment, away, manner, prefix, hist, pertaining, sion, signifies, english, person, shakesp, form, able, one's, ital, subst, make, used, without, term",
 4: 'english, sounds, pronunciation, sound, french, latin, words, alphabet, vowel, language, phonetic, forms, greek, vowels, german, form, languages, used, also, letters, word, represented, spell

In [168]:
def get_doctopics_df(lda_model, force=False):
    fn='data.gensim.doctopics.all.pkl'
    if not force and os.path.exists(fn): 
        return pd.read_pickle(fn)

    doctopics=[]
    for d in iter_corpus(max_pages_per_work=None):
    # for d in iter_corpus():
        toks = d.pop('page_toks')
        bow = dictionary.doc2bow(toks)
        topicscores = dict(lda_model.get_document_topics(bow))
        odx = {
            **d,
            'page_len':len(toks), 
            **{f'T{ti:02}':tv for ti,tv in topicscores.items()}
        }
        doctopics.append(odx)
    doctopicdf=pd.DataFrame(doctopics).fillna(0).set_index(['work_cluster','work_id','page_id','page_len'])
    doctopicdf = doctopicdf[sorted(doctopicdf.columns)]
    doctopicdf.to_pickle(fn)
    return doctopicdf

In [169]:
doctopicdf = get_doctopics_df(lda_model, force=True)

Iterating over jsonl: 100%|██████████| 2097745/2097745 [2:36:36<00:00, 223.25it/s]     


In [173]:
dfclusteravg = doctopicdf.groupby('work_cluster').mean()
dfclustermeta = df_metadata.drop_duplicates('cluster_id_s').set_index('cluster_id_s')
dfclustermeta['pub_date'] = pd.to_numeric(dfclustermeta['pub_date'],errors='coerce')
dfclustermeta['pub_date_dec'] = dfclustermeta.pub_date // 10 * 10
dfclusters = dfclustermeta.join(dfclusteravg).reset_index().set_index(list(dfclustermeta.reset_index().columns))
dfclusters

,,,,,,,,,,,,,,,,,,,,,,,,,T00,T01,T02,T03,T04,T05,T06,T07,T08,T09,...,T40,T41,T42,T43,T44,T45,T46,T47,T48,T49
cluster_id_s,source_id,group_id_s,source_t,source_url,title,sort_title,pub_date,pub_place,publisher,enumcron,author,author_exact,collections,collections_exact,item_type,order,work_type_s,last_modified,_version_,first_page_i,book_journal_s,subtitle,notes,pub_date_dec,,,,,,,,,,,,,,,,,,,,,
dul1.ark:/13960/t6d23816n,dul1.ark:/13960/t6d23816n,dul1.ark:/13960/t6d23816n,['HathiTrust'],https://hdl.handle.net/2027/dul1.ark:/13960/t6d23816n,"Lyrical ballads, with other poems: in two volumes.","Lyrical ballads, with other poems: in two volumes.",1802.0,Philadelphia,James Humphreys,v.1-2,"Wordsworth, William, 1770-1850","Wordsworth, William, 1770-1850",['Uncategorized'],['Uncategorized'],work,0,full-work,2023-02-02T15:00:45.957Z,1756731849797795847,,,,,1800.0,0.003424,0.004864,0.000728,0.001658,0.000124,0.000775,0.001543,0.000384,0.020090,0.003110,...,0.509232,0.002927,0.012918,0.009641,0.000436,0.002785,0.000247,0.002251,0.041251,0.001637
dul1.ark:/13960/t6d23816n-p11,dul1.ark:/13960/t6d23816n,dul1.ark:/13960/t6d23816n-p11,['HathiTrust'],https://hdl.handle.net/2027/dul1.ark:/13960/t6d23816n,Preface [to Lyrical Ballads],Preface [to Lyrical Ballads],1802.0,Philadelphia,James Humphreys,v.1,"Wordsworth, William.","Wordsworth, William.","['Literary', 'Original Bibliography']","['Literary', 'Original Bibliography']",work,0,excerpt,2023-02-14T19:59:25.780Z,1757837803736006658,11.0,"Lyrical ballads, with other poems: in two volumes.",,,1800.0,0.031504,0.023285,0.000000,0.003057,0.000000,0.000000,0.000000,0.001404,0.103076,0.000000,...,0.032716,0.024029,0.008951,0.003369,0.000000,0.000000,0.001435,0.000000,0.000604,0.002256
osmondmystical,inu.32000006890968,inu.32000006890968,['HathiTrust'],https://hdl.handle.net/2027/inu.32000006890968,"The mystical poets of the English church,","mystical poets of the English church,",1919.0,London,Society for Promoting Christian Knowledge,,"Osmond, Percy H. (Percy Herbert), b. 1872","Osmond, Percy H. (Percy Herbert), b. 1872",['Literary'],['Literary'],work,0,full-work,2023-02-14T19:34:42.875Z,1757836248763858945,,,,,1910.0,0.003562,0.039340,0.000530,0.002761,0.001073,0.000351,0.001522,0.001280,0.021738,0.001340,...,0.314318,0.026703,0.019021,0.026514,0.001040,0.002259,0.000773,0.004191,0.007594,0.002965
townanalysis,hvd.hn5bic,hvd.hn5bic,['HathiTrust'],https://hdl.handle.net/2027/hvd.hn5bic,An analysis of the derivative words in the English language :,"analysis of the derivative words in the English language : or, A key to their precise analytic definitions, by prefixes and suffixes ... /",1841.0,New York,Clement & Packard,,"Town, Salem, 1779-1864","Town, Salem, 1779-1864","['Linguistic', 'Word Lists']","['Linguistic', 'Word Lists']",work,0,full-work,2023-02-14T19:59:31.530Z,1757837809744347137,,,"or, A key to their precise analytic definitions, by prefixes and suffixes ...",,1840.0,0.138243,0.004509,0.000960,0.187288,0.009098,0.013130,0.010137,0.031388,0.066631,0.005473,...,0.043227,0.012480,0.046447,0.006603,0.003084,0.002100,0.007035,0.000974,0.084179,0.002706
CW0100721039,CW0100721039,CW0100721039,['Gale'],https://link.gale.com/apps/doc/CW0100721039/ECCO?u=prin77918&sid=gale_api&xid=73b8a5b3,A new and general biographical dictionary,new and general biographical dictionary containing an historical and critical account of the lives and writings of the most eminent persons in every nation; particularly the British and Irish; ...,1798.0,London,"G.G. and J. Robinson, J. Johnson, J. Nichols, J. Sewell, H.L. Gardner [and 23 others in London]",14,,,['Literary'],['Literary'],work,0,full-work,2023-02-14T19:34:42.875Z,1757836248776441856,,,containing an historical and critical account of the lives and writings of the most eminent persons in every nation; particularly the British and Irish; ...,,1790.0,0.005753,0.074763,0.000960,0.001974,0.000433,0.000000,0.001350,0.007532,0.156724,0.005594,...,0.008047,0.001306

In [174]:
import altair as alt

COLLECTIONS = {
    # 'Dictionaries',
    'Linguistic',
    'Literary',
    # 'Uncategorized'
}

def plot_topic(tnum):
    key=f'T{tnum:02}'
    figdf=dfclusters.reset_index()[['cluster_id_s','title','pub_date', 'collections', key]]
    figdf['collections'] = figdf['collections'].apply(lambda x: ''.join(y for y in x.split(',')[0].strip() if y.isalpha() or y==' '))
    figdf = figdf[figdf.collections.isin(COLLECTIONS)]
    figdf.columns = ['cluster', 'title', 'date', 'genre', 'topic']
    figdf = figdf[1700<=figdf.date]
    
    fig = alt.Chart(figdf).mark_circle(size=60).encode(
        x = alt.X('date', scale=alt.Scale(domain=[1700, 1920])),
        y='topic',
        color='genre',
        tooltip=figdf.columns.tolist()
    ).interactive(
    ).properties(
        width=800,
        height=400,
    # ).facet(
    #     facet='genre:N',
    #     columns=2,
    # ).resolve_scale(
    #     y='independent'
    ).properties(
        title = f'Topic {tnum}: {topicwords[tnum][:100]}'
    )
    return fig

In [175]:
plot_topic(23)

alt.Chart(...)